In [29]:

import bs4

import requests

In [2]:
r = requests.get('https://www.pref.osaka.lg.jp/iryo/osakakansensho/happyo.html')

In [4]:
soup = bs4.BeautifulSoup(r.content, 'html.parser')
soup

<?xml version="1.0" encoding="shift_jis" standalone="yes"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="大阪府新型コロナウイルス感染患者,新型コロナ患者発生状況,新型コロナ報道発表" name="keywords"/>
<meta content="新型コロナウイルス感染症患者の発生状況について" property="og:title"/>
<meta content="article" property="og:type"/>
<meta content="http://www.pref.osaka.lg.jp/iryo/osakakansensho/happyo.html" property="og:url"/>
<meta content="http://www.pref.osaka.lg.jp/img/ogp_image.gif" property="og:image"/>
<meta content="大阪府" property="og:site_name"/>
<meta content="新型コロナウイルス感染症患者の発生状況について 大阪府における新型コロナウイルス感染症に関する患者の発生状況や検査

In [5]:
link = soup.find_all('a')

# getでリンク要素のhref属性の値を取得して出力

In [6]:
lst = []
for l in link:
    
# getでリンク要素のhref属性の値を取得して出力
    h = l.get('href')
    lst.append(h)

In [7]:
import re

In [8]:

links = soup.find_all('a', attrs={ 'href': re.compile(r'\d{4}\.pdf$') })
for link in links:
    pdf_link = link.get('href')
    

In [9]:
header = 'https://www.pref.osaka.lg.jp/'

In [10]:
url_for_pdf = header + pdf_link

In [11]:
import datetime as dt

In [12]:
m = dt.datetime.today().strftime('%m')

In [13]:
d = dt.datetime.today().strftime('%d')

In [14]:
date_filename = m + d

In [15]:
r = requests.get(url_for_pdf)  
with open(f'{m + d}.pdf', 'wb') as f:
    f.write(r.content)


In [16]:
import tabula

In [17]:
t = tabula.read_pdf(input_path= f'{date_filename}.pdf',
                    pages=2,
                    area=(68, 112, 93, 357))

In [18]:
import pandas as pd

In [19]:
_df = pd.DataFrame(t[0].columns.array)#.set_index['感染者数', '累計', '男性', '女性', '調査中']

In [20]:
_df.index = ['陽性者数', '累計', '男性', '女性', '調査中']

In [21]:
_df.columns= [date_filename]

In [22]:
t2 = tabula.read_pdf(f'{date_filename}.pdf',
                        pages=2,
                        area=(188, 150, 233, 212))

In [23]:
t2 = t2[0]

In [24]:
t2.index = ['総数', 'PCR', '抗原検査']

In [25]:
posi_cases = _df.loc['陽性者数', f'{date_filename}']

In [26]:
total_n = t2.loc['総数', '検査件数'].replace(',','')

In [27]:
pos_rate = (int(posi_cases) / int(total_n))

In [28]:
pos_rate2= '{:.2%}'.format(pos_rate)

In [ ]:
### ---- 各市町村個別情報を取得

t3=tabula.read_pdf(f'{date_filename}.pdf',
                pages=3,#３ぺーじめを読む
                area=(58, 55, 302.72, 415.83),#エリア指定
                columns=(57, 111.94, 168.18, 224.94, 302.19, 357.90))#列指定

In [ ]:
t4 = t3[0].dropna(axis=1)

In [ ]:
city_cases1 = t4.loc[:, ['市町村', '発生者数', '累計']]

In [ ]:
city_cases2 = t4.loc[:, ['市町村.1', '発生者数.1', '累計.1']]
city_cases2.columns = ['市町村', '発生者数', '累計']

In [ ]:
city_cases_conc = pd.concat([city_cases1, city_cases2])

In [ ]:
city_cases_conc = city_cases_conc.reset_index(drop=True)

In [ ]:
city_cases_conc = city_cases_conc.replace(',','')

In [ ]:
d = []
for i in city_cases_conc['累計']:
    r = i.replace(',','')
    d.append(int(r))

In [ ]:
city_cases_conc['累計'] = d

In [ ]:
個別累計 = city_cases_conc.累計.sum()

In [ ]:
累計 = int(_df.loc['累計',:].values[0].replace(',', ''))

In [ ]:
city_cases_conc[f'更新日'] = date_filename

In [ ]:
city_cases_conc.to_csv('累計.csv',
                       encoding='utf_8_sig',
                       mode='a',
                       index=False,
                       header=None)

In [ ]:
_df_T = _df.T

In [ ]:
_df_T.to_csv('陽性者数.csv', 
             encoding='utf_8_sig',
             mode='a',
             header=None)